# Scraping code takes a list of countries and returns travel advisories from travel.state.gov

## "country list" (below) should be replaced with input list from users, perhaps uploaded from csv

In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd


# Travel Advisories


In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
country_list = ['France', 'Australia', 'China', 'New Zealand', 'Gabon']

In [4]:
airport_list = ['CDG', 'SYD', 'PEK', 'AKL', 'LBV']

In [5]:
country_date = []
country_level = []
country_report = []
country_url = []


In [5]:
# # TESTING download of results and parsing
# url = 'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/'
# # url = url.replace(" ","")
# browser.visit(url)
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')
# banner = soup.find('div', class_= 'tsg-rwd-csi-travel-advisories')
# print(banner)

None


In [ ]:
# date = banner.find('div', class_='tsg-rwd-eab-date-frame').text.strip()
# print(date)

In [ ]:
# level = banner.find('h3', class_='tsg-rwd-eab-title-frame').text.strip()
# print(level)

In [ ]:
# report = banner.find('div', class_='tsg-rwd-alert-teaser').span.text.strip()
# print(report)

In [6]:
for x in range(len(country_list)):
    url = 'https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/' + country_list[x] + '.html'
    url = url.replace(" ","")
    country_url.append(url)
    print(url)
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    banner = soup.find('div', class_= 'tsg-rwd-csi-travel-advisories')
    date = banner.find('div', class_='tsg-rwd-eab-date-frame').text.strip()
    country_date.append("Last advisory date: " + date)
    level = banner.find('h3', class_='tsg-rwd-eab-title-frame').text.strip()
    country_level.append(level)
    report = banner.find('div', class_='tsg-rwd-alert-teaser').span.text.strip()
    country_report.append(report)


https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/France.html
https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/Australia.html
https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/China.html
https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/NewZealand.html
https://travel.state.gov/content/travel/en/international-travel/International-Travel-Country-Information-Pages/Gabon.html


In [19]:
#Creating a single list of lists for use in other code / javascript
summary_report = [country_level, country_date, country_report]
summary_report

[['France - Level 2: Exercise Increased Caution',
  'Australia - Level 1: Exercise Normal Precautions',
  'China - Level 4: Do Not Travel',
  'New Zealand - Level 1: Exercise Normal Precautions',
  'Gabon - Level 1: Exercise Normal Precautions'],
 ['Last advisory date: April 1, 2019',
  'Last advisory date: January 21, 2020',
  'Last advisory date: February 2, 2020',
  'Last advisory date: November 21, 2019',
  'Last advisory date: January 10, 2019'],
 ['Exercise increased caution in France due to terrorism and civil unrest.',
  'Exercise normal precautions in Australia. Areas affected by bushfires have increased risk. Read the entire Travel Advisory.',
  'Do not travel to China due to the novel coronavirus first identified in Wuhan, China. On January 30, the World Health Organization (WHO) determined the rapidly spreading outbreak constitutes a Public Health Emergency of International Concern (PHEIC). Travelers should be prepared for the possibility of travel restrictions with little 

In [21]:
#Testing print summary, using indexing in the list of lists
print(summary_report[0][2])
print(summary_report[1][2])
print(summary_report[2][2])

China - Level 4: Do Not Travel
Last advisory date: February 2, 2020
Do not travel to China due to the novel coronavirus first identified in Wuhan, China. On January 30, the World Health Organization (WHO) determined the rapidly spreading outbreak constitutes a Public Health Emergency of International Concern (PHEIC). Travelers should be prepared for the possibility of travel restrictions with little or no advance notice. Most commercial air carriers have reduced or suspended routes to and from China.


In [12]:
for summaries in zip(country_level, country_date, country_report):
    print(summaries)

('France - Level 2: Exercise Increased Caution', 'Last advisory date: April 1, 2019', 'Exercise increased caution in France due to terrorism and civil unrest.')
('Australia - Level 1: Exercise Normal Precautions', 'Last advisory date: January 21, 2020', 'Exercise normal precautions in Australia. Areas affected by bushfires have increased risk. Read the entire Travel Advisory.')
('China - Level 4: Do Not Travel', 'Last advisory date: February 2, 2020', 'Do not travel to China due to the novel coronavirus first identified in Wuhan, China. On January 30, the World Health Organization (WHO) determined the rapidly spreading outbreak constitutes a Public Health Emergency of International Concern (PHEIC). Travelers should be prepared for the possibility of travel restrictions with little or no advance notice. Most commercial air carriers have reduced or suspended routes to and from China.')
('New Zealand - Level 1: Exercise Normal Precautions', 'Last advisory date: November 21, 2019', 'Exerci

In [14]:
reports = summaries[0]
print(reports)

Gabon - Level 1: Exercise Normal Precautions


# Get City GeoJSON

In [10]:
air = pd.read_csv('airport-codes_csv.csv')
air.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [15]:
air['iso_country'].value_counts()

US    22885
BR     4545
CA     2792
AU     1945
KR     1374
      ...  
LI        1
GM        1
BL        1
VA        1
NU        1
Name: iso_country, Length: 242, dtype: int64

In [18]:
for x in range(len(airport_list)):
    if x == air['iata_code']:
        print(air(['name'], ['coordinates']))


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().